In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math



In [3]:
data_basic = './data/gender_race_admission_type_charttime_fio2_plateaupressure.csv'
data_basic2 = './data/basic_data2.csv'
data_O2 = './data/O2_flow.csv'
data_vet = './data/ventailation_data.csv'

data_basic_datalist = pd.read_csv(data_basic)
data_basic2_datalist = pd.read_csv(data_basic2)
data_O2_datalist = pd.read_csv(data_O2)
data_vet_datalist = pd.read_csv(data_vet)

df = data_vet_datalist
df['charttime'] = pd.to_datetime(df['charttime'])
df = df.sort_values(by=['stay_id', 'charttime'])
df = df.set_index('charttime')
df_resampled = df.groupby('stay_id').resample('H').max()
df_resampled['respiratory_rate_set'] = df_resampled['respiratory_rate_set'].fillna(method='ffill')
df_resampled['tidal_volume_observed'] = df_resampled['tidal_volume_observed'].fillna(method='ffill')
df_resampled['peep'] = df_resampled['peep'].fillna(method='ffill')
df_resampled['minute_ventilation'] = df_resampled['minute_ventilation'].fillna(method='ffill')
df_resampled = df_resampled.drop(columns='stay_id')
df_resampled = df_resampled.reset_index()
df_resampled['subject_id'] = df_resampled.groupby('stay_id')['subject_id'].fillna(method='ffill')


df1 = data_O2_datalist
df1['charttime'] = pd.to_datetime(df1['charttime'])
df1 = df1.sort_values(by=['stay_id', 'charttime'])
df1 = df1.set_index('charttime')
df_resampled1 = df1.groupby('stay_id').resample('H').max()
df_resampled1['O2_flow'] = df_resampled1['O2_flow'].fillna(method='ffill')
df_resampled1 = df_resampled1.drop(columns='stay_id')
df_resampled1 = df_resampled1.reset_index()
df_resampled1['subject_id'] = df_resampled1.groupby('stay_id')['subject_id'].fillna(method='ffill')

df2 = data_basic_datalist
df2['charttime'] = pd.to_datetime(df2['charttime'])
df2 = df2.sort_values(by=['stay_id', 'charttime'])
df2 = df2.set_index('charttime')
df_resampled2 = df2.groupby('stay_id').resample('H').max()
df_resampled2['gender'] = df_resampled2['gender'].fillna(method='ffill')
df_resampled2['race'] = df_resampled2['race'].fillna(method='ffill')
df_resampled2['admission_type'] = df_resampled2['admission_type'].fillna(method='ffill')
df_resampled2['fio2'] = df_resampled2['fio2'].fillna(method='ffill')
df_resampled2['plateau_pressure'] = df_resampled2['plateau_pressure'].fillna(method='ffill')
df_resampled2 = df_resampled2.drop(columns='stay_id')
df_resampled2 = df_resampled2.drop(columns='hadm_id')
df_resampled2 = df_resampled2.reset_index()
df_resampled2['subject_id'] = df_resampled2.groupby('stay_id')['subject_id'].fillna(method='ffill')

df3 = data_basic2_datalist
df_merged = pd.merge(df_resampled2, df3, on=['stay_id', 'subject_id'], how='inner')
df_merged = pd.merge(df_merged, df_resampled1, on=['stay_id', 'subject_id', 'charttime'], how='inner')
df_merged = pd.merge(df_merged, df_resampled, on=['stay_id', 'subject_id', 'charttime'], how='inner')
df_temp = df_merged
df_merged = df_merged.drop(columns=['subject_id'])
df_merged.insert(0, 'subject_id', df_temp['subject_id'])
stay_id_30005707  = df_merged[df_merged['stay_id'] == 30005707]
df_str = stay_id_30005707.to_string(index=False)

KeyboardInterrupt: 

In [13]:
data_GT = './data/ground_truth_meta_data.csv'
data_GT_datalist = pd.read_csv(data_GT)
df_str = data_GT_datalist.to_string(index=False)
df4 = pd.DataFrame()
df4['subject_id']= data_GT_datalist['subject_id']
df4['stay_id'] = data_GT_datalist['stay_id']
df4['label'] = data_GT_datalist['label']
df4 = df4.drop_duplicates()
df_str = df4.to_string(index=False)
print(df4)

0                                                   14569364
1                                                   18756985
5                                                   16828280
7                                                   11365767
8                                                   16623708
                                 ...                        
4887                                                19962126
4888                                                10128085
4889                                                11539827
stay_id    0       30005707
1       30006983
2       3000...
label      0       1
1       0
2       0
3       0
4     ...
Name: subject_id, Length: 2842, dtype: object


C:\Users\jerryshu\AppData\Local\Temp\ipykernel_7692\2531855681.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['stay_id'] = data_GT_datalist['stay_id']


In [12]:
for line in df_str.split('\n'):
    print(line)

                                          14569364
                                          18756985
                                          16828280
                                          11365767
                                          16623708
                                          16815463
                                          13771243
                                          10191971
                                          15895725
                                          12875089
                                          11636169
                                          17232630
                                          14880575
                                          19518348
                                          12006413
                                          17057610
                                          19935817
                                          12835259
                                          14981236
                               

In [4]:
merge_data = './data/merged_data.csv'
df_merged.to_csv(merge_data, index=False)